# GADMA Tutorial

## Outline
1. Our Dataset
2. Overview of GADMA
3. Example Run of GADMA
4. Hands On

## Getting Help after Tutorial
- Contact: [enoskova.me](http://enoskova.me)
- Feel free to send me [email](mailto:ekaterina.e.noskova@gmail.com)
- [Getting help for GADMA](https://github.com/ctlab/GADMA#getting-help)

## 1.  Our Dataset

We have a dataset for clouded leopards (*Neofelis nebulosa*). We have data for 5 diploid individuals (10 samples).

All the data is available in the `data` folder:

In [ ]:
%%bash
tar -xvzf data.tar.qz
ls data

- File `data/clouded_leopard_data.vcf` is our VCF file for all 5 individuals (X chromosome is excluded):

In [ ]:
%%bash
# First five lines
head -10 data/clouded_leopard_data.vcf

In [ ]:
%%bash
# Our contigs
head -7 data/clouded_leopard_data.vcf | tail -4

In [ ]:
%%bash
# The header line of the VCF file (we take first 84 lines of file and then show the last one)
head -9 data/clouded_leopard_data.vcf | tail -1

- File `data/popmap` provides population assignments per individual (all our individuals are from the same population that is marked as `NN`):

In [ ]:
%%bash
cat data/popmap

- The length of the given sequence is equal to $118{,}000{,}000$ bp.

We will also use the following values associated with our species of interest [\[Bursell et al. 2022\]](https://doi.org/10.1016/j.isci.2022.105647):
- Generation time 7 years
- Mutation rate of $2.22 \times 10^{-9}$ per base pair per year = $1.554 \times 10^{-8}$ per base pair per **generation**
- Recombination rate of $1$ cM/Mb

## Run easySFS to build SFS

In [ ]:
%%bash
#../2_easySFS_tutorial/easySFS/easySFS.py -i data/clouded_leopard_data.vcf -p data/popmap -a --unfolded --total-length 118000000 -o outputs/easySFS_output -f --proj 10 

## Picture of our SFS

In [ ]:
from scripts.draw_sfs import draw_1d_sfs
draw_1d_sfs("outputs/easySFS_output/dadi/NN-10.sfs")

-----
## 2. Overview of GADMA

### What is [GADMA](https://github.com/ctlab/GADMA)?
- Tool for demographic inference from the genetic data.
- Provides demographic history that has highest value of likelihood with the data.
- Includes several likelihood components of existing tools under the common interface (`dadi`, `moments`, `momi2`, `momentsLD`)
- Has effective global optimization for parameter search.
- New model specification using *structure*.
- Repeats demographic inference and provides best result among several repeats (30 repeats is good, 100 is better).

Papers to cite: [\[Noskova et al. 2020\]](https://doi.org/10.1093/gigascience/giaa005), [\[Noskova et al. 2023\]](https://doi.org/10.1101/2022.06.14.496083) and other papers regarding used engines (see more [here](https://gadma.readthedocs.io/en/latest/citations.html)). 

### What is the difference between likelihood engines in GADMA?

GADMA has several engines to evaluate likelihood of the demographic history and data. More information about each engine could be found in the [documentation](https://gadma.readthedocs.io/en/latest/user_manual/set_engine.html).

<img src="pictures/Screenshot_engines.png" width="700" align="left"/>

### What is the model of the demographic history?

Classical tools for demographic inference require parameterized model of the demographic history. It should be specified using interface of the tool and could be error-prone. For example, let us consider the following model for the demographic history of two populations:
- Ancestral population had size equal to $N_{anc}$.
- Ancestral population experienced instanstenious growth up to ($nu_{1F}\cdot N_{anc}$) individuals and it has constant size for ($2\cdot T_p\cdot N_{anc}$) generations.
- Then second population diverged from ancestral population.
- Second population experienced exponential growth from ($nu_{2B}\cdot N_{anc}$) up to ($nu_{2F}\cdot N_{anc}$) individuals during ($2\cdot T\cdot N_{anc}$) generations after split.
- Size of the first population was equal to the size of ancestral population.
- There was migration between populations with rate equal to $m / (2\cdot N_{anc})$ migrants per generation.

This model has one parameter for $N_{anc}$ and additional 6 parameters. Example of the model specification for `dadi` using its Python API:

In [ ]:
def prior_onegrow_mig(params, ns, pts):
    """
    Model with growth, split, bottleneck in pop2, exp recovery, migration

    params list is
    nu1F: The ancestral population size after growth. (Its initial size is
          defined to be 1.)
    nu2B: The bottleneck size for pop2
    nu2F: The final size for pop2
    m: The scaled migration rate
    Tp: The scaled time between ancestral population growth and the split.
    T: The time between the split and present

    ns = (n1,n2): Size of fs to generate.
    pts: Number of points to use in grid for evaluation.
    """
    nu1F, nu2B, nu2F, m, Tp, T = params
    n1,n2 = ns
    # Define the grid we'll use
    xx = yy = dadi.Numerics.default_grid(pts)

    # phi for the equilibrium ancestral population
    phi = dadi.PhiManip.phi_1D(xx)
    # Now do the population growth event.
    phi = dadi.Integration.one_pop(phi, xx, Tp, nu=nu1F)

    # The divergence
    phi = dadi.PhiManip.phi_1D_to_2D(xx, phi)
    # We need to define a function to describe the non-constant population 2
    # size. lambda is a convenient way to do so.
    nu2_func = lambda t: nu2B*(nu2F/nu2B)**(t/T)
    phi = dadi.Integration.two_pops(phi, xx, T, nu1=nu1F, nu2=nu2_func, 
                                    m12=m, m21=m)

    # Finally, calculate the spectrum.
    sfs = dadi.Spectrum.from_phi(phi, (n1,n2), (xx,yy))
    return sfs

### Model specification using structure

GADMA requieres number of epochs in the demographic history. It has several additional handlers for flexibility:
- Enable/disable migrations
- Enable/disable selection
- Enable/disable dynamics of population size

More information [here](https://gadma.readthedocs.io/en/latest/user_manual/set_model/set_model_struct.html).

<img src="pictures/gadma.png" width="700" align="left"/>

### Dynamics of population size change

GADMA has flexible dynamics of population size change for model with structure.

Population dynamic can be:

- Constant (Sudden)
- Linear
- Exponential

<img src="pictures/sudden.png" width="150" align="left"/>
<img src="pictures/linear.png" width="150" align="left"/> 
<img src="pictures/exponential.png" width="150" align="left"/>

### Notes about structure

- Structure model could be used only for data of either 1, 2 or 3 populations. For 4 and 5 populations usual models could be used (custom models).
- Structure should not be very complex: \[3\] for one population, \[2, 1\] for two populations and \[2, 1, 1\] for three populations is enough.
- For three populations the most ancient population should be specified (in order to know the tree topology). It could be just sorted through all options.
- GADMA has a special inference scheme when the structure is increased: it starts from the simple initial structure and then use more complex structures until reachs the final one. Please, use this option as it provides better estimations.


--------
## 3. Example Run of GADMA

### Installation

[Documentation on installation](https://gadma.readthedocs.io/en/latest/user_manual/installation.html#installing-the-latest-release)

GADMA can be easily installed via `pip` or `conda`. Some of its engines (`momi2`) should be installed manually.



In [ ]:
%%bash
pip install gadma

### Run GADMA with the `--help` option

In [ ]:
%%bash
gadma --help

### GADMA output

[More information about GADMA output in the documentation](https://gadma.readthedocs.io/en/latest/user_manual/output.html)

Before we run GADMA let us take a look at its output. GADMA was already run and it created an output directory.
- Our run was saved in `outputs/gadma_outputs/gadma_output_2epochs`:

In [ ]:
%%bash
ls outputs/gadma_outputs/gadma_output_2epochs/

- All command line output is saved in file `GADMA.log`:

In [ ]:
%%bash
head -17 outputs/gadma_outputs/gadma_output_2epochs/GADMA.log

In [ ]:
%%bash
tail -33 outputs/gadma_outputs/gadma_output_2epochs/GADMA.log

- GADMA uses parameters file with specified options. Example of parameters file could be found [here](https://gadma.readthedocs.io/en/latest/user_manual/example_params_file.html).

In [ ]:
%%bash
cat outputs/gadma_outputs/gadma_output_2epochs/params_file

- Each repeat of GADMA run is saved in its own folder:

In [ ]:
%%bash
ls outputs/gadma_outputs/gadma_output_2epochs/1

### Run GADMA for one-epoch demographic history inference

Let us run GADMA for our data. We need to specify params_file for GADMA with options. We will use moments in our example, but you can choose either `dadi` or `momi2` as well (they are also SFS-based engines).

In [ ]:
%%bash
cat gadma_params_files/gadma_params_file_1epoch

In [ ]:
%%bash
rm -rf outputs/gadma_outputs/gadma_output_1epoch
gadma -p gadma_params_files/gadma_params_file_1epoch

### Run GADMA for two-epoch demographic history inference

In [ ]:
%%bash
cat gadma_params_files/gadma_params_file_2epochs

In [ ]:
%%bash
rm -rf outputs/gadma_outputs/gadma_output_2epochs
gadma -p gadma_params_files/gadma_params_file_2epochs

### Run GADMA for two-epoch demographic history inference with wide bounds

Sometimes GADMA prints:
```text
INFO: Some parameters of the best model hit their bounds: nu11 hit lower bounds
```

The bounds for the parameters are the following (they are located in the `extra_params_file`, more information is [here](https://gadma.readthedocs.io/en/latest/user_manual/extra_params_file.html)):
- Minimum population size `min_N` = $0.01 (\cdot N_{anc})$
- Maximum population size `max_N` = $100 (\cdot N_{anc})$
- Minimum epoch time `min_T` = ~$0$
- Maximum epoch time `max_T` = $5 (\cdot 2 \cdot N_{anc})$
- Minimum migration rate `min_m` = $0$
- Maximum migration rate `max_m` = $10 (/ (2\cdot N_{anc}))$

All values for `dadi` and `moments` are in genetic units. We should check the parameters in output file with code. We are looking for `p0`:

In [ ]:
%%bash
head -20 outputs/gadma_outputs/gadma_output_2epochs/best_logLL_model_moments_code.py

In [ ]:
%%bash
cat gadma_params_files/gadma_params_file_2epochs_wide_bounds

In [ ]:
%%bash
rm -rf outputs/gadma_outputs/gadma_output_2epochs_wide_bounds
gadma -p gadma_params_files/gadma_params_file_2epochs_wide_bounds

### Run GADMA for three-epoch demographic history inference using previous runs

In [ ]:
%%bash
cat gadma_params_files/gadma_resume_3epochs

In [ ]:
%%bash
gadma --resume outputs/gadma_outputs/gadma_output_2epochs_wide_bounds \
-p gadma_params_files/gadma_resume_3epochs \
-o outputs/gadma_outputs/gadma_output_3epochs

### Reconstruct the Likelihood

We can obtain the likelihood of the best result by running generated code from GADMA:

In [ ]:
%%bash
python outputs/gadma_outputs/gadma_output_3epochs/best_logLL_model_moments_code.py

---
## 4. Hands On!

Repeat this tutorial on your computer and obtain the results. Please send your results to us [here](https://forms.gle/cs6xY7y9pUA3AaBH7).

You can change run options if you like:
* Use different engine: e.g. `dadi`
* Use time for one generation of 7 years
* Any other option